In [3]:
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.vectorstores import FAISS
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings


In [7]:
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Maran\StudyMaterials\Git\LangChain\Semantic_Search\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Maran\StudyMaterials\Git\LangChain\Semantic_Search\venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Darshan\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you eit

In [33]:
documents = ["First document about machine learning",
             "Second document about cloud computing",
             "Third document about architecture with azure"]
faiss_index = FAISS.from_texts(documents, embedding=embedding)



In [14]:
llm = OpenAI(temperature=0.9)
prompt_template =""" 
Given the followingg documents {documents} and the query {query}, please provide the details"""
prompt = PromptTemplate(
    input_variables=["documents", "query"],
    template=prompt_template
)

In [35]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [31]:
from langchain.chains import LLMChain
llm_chain = LLMChain(llm=llm, prompt=prompt)
query ="What is the role of azure in cloud computing?"


In [42]:
results = faiss_index.similarity_search_with_score(query)
for doc, score in results:
    print(f"Content: {doc.page_content}, \nMetadata: {doc.metadata}, \nScore: {score}")

Content: Third document about architecture with azure, 
Metadata: {}, 
Score: 0.822560727596283
Content: Second document about cloud computing, 
Metadata: {}, 
Score: 0.9255028367042542
Content: First document about machine learning, 
Metadata: {}, 
Score: 1.7017558813095093


In [43]:
metadata =[{"category":"AI"},
           {"category":"Cloud"},
           {"category":"Architecture"}]
faiss_index_with_metadata = FAISS.from_texts(documents, embedding, metadatas=metadata)

In [46]:
filter = dict(category="Cloud")
filter

{'category': 'Cloud'}

In [47]:
results = faiss_index_with_metadata.similarity_search_with_score(query,filter=filter)
for doc in results:
    print(f"Content: {doc[0].page_content}, \nMetadata: {doc[0].metadata}\n\n")

Content: Second document about cloud computing, 
Metadata: {'category': 'Cloud'}


